In [242]:
#####################################################################################
run_models = True
run_k_fold = False
overleaf_metrics = False
print_all = False
is_deadline = False
#####################################################################################



####################################################################################
# ground_truth_method = 'hist'
# ground_truth_method = 'ecdf'

two_labels = True
break_window_features = True
####################################################################################




#####################################################################################
k_fold_n_splits = 5
#####################################################################################



#####################################################################################
# arousal_signals = ['PP']
# arousal_signals = ['PP', 'BR']
# arousal_signals = ['PP_HR_BR']
# arousal_signals = ['PP_HR', 'HR_BR', 'PP_BR']


# arousal_signals = ['PP', 'HR', 'BR', 'PP_HR_BR']
arousal_signals = ['PP', 'HR', 'BR', 'PP_HR', 'HR_BR', 'PP_BR', 'PP_HR_BR']
#####################################################################################



#####################################################################################
%run us_common_functions.ipynb
#####################################################################################



#####################################################################################
# ['DNN', 'LSTM']
model_name = 'LSTM'
#####################################################################################




# ####################################################################################
# ### study_combinations_to_run = list(study_combinations.keys())
# ### study_combinations_to_run = list(study_combinations.keys())[:4]
# ### study_combinations_to_run = list(study_combinations.keys())[4:]

# ### study_combinations_to_run = ['sim1', 'sim1___sim2___tt1___office_tasks']
# ### study_combinations_to_run = ['sim2', 'tt1', 'office_tasks', 'sim1___sim2', 'sim1___sim2___tt1']
# ### study_combinations_to_run = ['sim1___sim2___tt1___office_tasks']
# ### study_combinations_to_run = ['sim1', 'tt1']
# ### study_combinations_to_run = ['sim1']
# ### study_combinations_to_run.remove(deadline_study)
# ###################################################################################




#####################################################################################
# All Features - Without Deadline Study
####################################################################################
four_studies = [sim1, sim2, tt1, office_tasks]
study_combinations, all_study_combinations = get_study_combinations(four_studies)
study_combinations_to_run = all_study_combinations


### study_combinations_to_run = all_study_combinations = all_study_combinations[:1]
### study_combinations_to_run = all_study_combinations[:-1]
### study_combinations_to_run = ['sim1___sim2___tt1___office_tasks']
### print(study_combinations_to_run)
####################################################################################




# #####################################################################################
# # For Deadline Study - Without BR
# #####################################################################################
# studies.append(deadline_study)
# study_combinations, all_study_combinations = get_study_combinations(studies)
# study_combinations_to_run = studies
# print(study_combinations_to_run)

# arousal_signals = ['PP', 'HR', 'PP_HR']
# model_features.remove('BR_Mean')
# model_features.remove('BR_SD')

# is_deadline=True
# ####################################################################################














####################################################################################
epochs = 10  # 10
batch_size = 32
verbose = 1
####################################################################################


In [243]:
####################################################################################
# Initialization
####################################################################################
model_metrics = {}
overleaf_final_tables = defaultdict(lambda : None)
study_combination_model_metrics = defaultdict(lambda : None)

# figure_path = all_studies_fig_dir
# plot_name = None






# ###################################################
# # data_6_rolling_w30_10_scaled
# dnn_data_file = 'data_6_rolling_w30_10_scaled.csv'

# dnn_window_size = dnn_data_file[-16:-11]
# dnn_window_size
# ###################################################


###################################################
# data_6_rolling_w15_5_scaled.csv
# data_6_rolling_w30_5_scaled.csv
# data_6_rolling_w60_5_scaled.csv
dnn_data_file = 'data_6_rolling_w30_5_scaled.csv'

dnn_window_size = dnn_data_file[-15:-11]
dnn_window_size
####################################################################################

'30_5'

In [244]:
all_studies_df = pd.read_csv(all_studies_data_dir + dnn_data_file)
# all_studies_df = pd.read_csv(all_studies_data_dir + dnn_data_file).iloc[:2000,]
all_studies_df.rename({'Perinasal_Window': 'PP_Window',
                       'Heart_Window': 'HR_Window',
                       'Breathing_Window': 'BR_Window',
                      }, axis=1, inplace=True)
all_studies_df.shape

(205110, 36)

In [245]:
all_studies_df.head(2)

,Study_Name,Subject,Study_Subject,Treatment,Age,NASA_Effort,NASA_Frustration,NASA_Mental,NASA_Performance,NASA_Physical,NASA_Temporal,NASA_Total,PP_Mean,PP_Median,PP_SD,PP_SS,PP_Window,HR_Mean,HR_Median,HR_SD,HR_SS,HR_Window,BR_Mean,BR_Median,BR_SD,BR_SS,BR_Window,PP_Arousal_Mode,HR_Arousal_Mode,BR_Arousal_Mode,PP_HR_Arousal_Mode,PP_BR_Arousal_Mode,HR_BR_Arousal_Mode,PP_HR_BR_Arousal_Mode,Gender,Gender_Male
0,sim1,2,sim1_2,CD,-0.886388,0.96301,0.332856,18.0,-0.204407,0.548524,0.367759,77.0,-0.704197,-0.043052,0.102389,0.064272,"0.0355228579351853, 0.0403412260359834, 0.0419...",0.919132,3.768182,-0.513985,518.068554,"3.5681818181818272, 4.968181818181819, 5.56818...",-1.088477,-4.366682,0.178627,607.649078,"-5.666681818181818, -5.586681818181818, -5.546...",non-arousal,neutral,non-arousal,non-arousal,non-arousal,neutral,non-arousal,0.691472,0
1,sim1,2,sim1_2,CD,-0.886388,0.96301,0.332856,18.0,-0.204407,0.548524,0.367759,77.0,-0.864359,-0.053195,-0.021171,0.112911,"0.0245498539848414, 0.0139063928086793, 0.0025...",0.698460,3.568182,0.052955,400.832190,"3.76818181818183, 2.5681818181818272, 2.168181...",-0.948112,-3.966682,0.281929,486.463558,"-5.386681818181819, -5.346681818181819, -5.286...",non-arousal,neutral,non-arousal,neutral,non-arousal,neutral,neutral,0.691472,0


In [246]:
all_studies_df.columns

Index(['Study_Name', 'Subject', 'Study_Subject', 'Treatment', 'Age',
       'NASA_Effort', 'NASA_Frustration', 'NASA_Mental', 'NASA_Performance',
       'NASA_Physical', 'NASA_Temporal', 'NASA_Total', 'PP_Mean', 'PP_Median',
       'PP_SD', 'PP_SS', 'PP_Window', 'HR_Mean', 'HR_Median', 'HR_SD', 'HR_SS',
       'HR_Window', 'BR_Mean', 'BR_Median', 'BR_SD', 'BR_SS', 'BR_Window',
       'PP_Arousal_Mode', 'HR_Arousal_Mode', 'BR_Arousal_Mode',
       'PP_HR_Arousal_Mode', 'PP_BR_Arousal_Mode', 'HR_BR_Arousal_Mode',
       'PP_HR_BR_Arousal_Mode', 'Gender', 'Gender_Male'],
      dtype='object')

In [247]:
all_studies_df = process_ts_features(all_studies_df)
all_studies_df = create_categorical_labels(all_studies_df)

In [248]:
################################################################################
# # window_cols = ['PP_Window']
# window_cols = ['PP_Window', 'HR_Window', 'BR_Window']

# for col in window_cols:
#     for i in range(len(all_studies_df[col][0])):
#         feature_name = col[:3] + str(int(dnn_window_size[:2])-i)
#         dnn_features.append(feature_name)
#         all_studies_df[feature_name] = [window[i] for window in all_studies_df[col]]
                                                   
# all_studies_df.columns
################################################################################

In [249]:
################################################################################
# dnn_features
################################################################################

In [250]:
start_time = time.time()

# study_combinations_to_run
# all_study_combinations
for study_combination in study_combinations_to_run:   
    
    train_studies = study_combinations[study_combination]['Train']
    test_studies = study_combinations[study_combination]['Test']
    # print(train_studies, test_studies, study_combination)
    
    train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
    # print(all_studies_df.shape, train_df.shape)
    
    model_metrics = {}

    
    for arousal_signal in arousal_signals:
        
        train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
        if two_labels: 
            train_df = train_df[train_df[arousal_signal+'_Arousal_Mode_Cat'] != 0]
        
        try:
            print(decorator + '\n' + decorator)
            print('Train Studies: ', study_combination.upper() + ' & Modality: ' + arousal_signal)
            print(decorator + '\n' + decorator + '\n')


            # Ref: https://stackoverflow.com/questions/3975376/understanding-dict-copy-shallow-or-deep
            model_metrics[arousal_signal] = {
                'K_Fold': copy.deepcopy(all_metrics),
                'Train': copy.deepcopy(all_metrics),
                'Test': {}
            }

            # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
            # print(train_df.shape)
            
            model = get_trained_dnn_model(train_df, arousal_signal)

            # for i, test_study in enumerate(test_studies):
            for j in range(1, len(test_studies)+1):
                for subset in itertools.combinations(test_studies, j):
              
                    test_study_subset = list(subset)
                    test_study_str = '___'.join(test_study_subset)
                    
                    if test_study_str in all_study_combinations:
                        # print('\n------------------------ Test Study' + str(i+1) + ': ' + test_study.upper() + ' ------------------------')
                        print('\n------------------------ Test Study: ' + test_study_str.upper() + ' ------------------------')

                        model_metrics[arousal_signal]['Test'][test_study_str.upper()] = copy.deepcopy(all_metrics)
                        # test_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(test_study_subset)]
                        test_df = all_studies_df.copy()[all_studies_df.Study_Name == test_study_str]
                        if two_labels:
                            test_df = test_df[test_df[arousal_signal+'_Arousal_Mode_Cat'] != 0]
                            # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
                            # print(test_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
                            
                        test_dnn_model(model, test_study_str.upper(), test_df, arousal_signal)

                        metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, test_study_str, model_metrics, k_fold=run_k_fold), ignore_index = True)

            if not test_studies: metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, None, model_metrics, k_fold=False), ignore_index = True)
    
        except Exception as e:
            print("An exception occurred for %s %s --> %s" % (study_combination, arousal_signal, e))
    
    # pprint.pprint(model_metrics)
    # overleaf_final_tables[study_combination] = get_overleaf_table(study_combination, model_metrics)
    study_combination_model_metrics[study_combination] = model_metrics
    
    
display(metrics_df)
metrics_file_name = 'dnn_deadline_' if is_deadline else 'dnn_'
metrics_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + metrics_file_name + dnn_window_size + '_v4_' + get_date_time() + '.csv', sep=',', index=False)


end_time = time.time()

----------------------------------------------------
----------------------------------------------------
Train Studies:  SIM1 & Modality: PP
----------------------------------------------------
----------------------------------------------------

Epoch 1/10
201/201 [==============================] - 8s 15ms/step - loss: -5.2029 - accuracy: 0.0045 - auc_480: 0.0000e+00 - f1_m: 0.0133 - precision_m: 0.0556 - recall_m: 0.0094 - val_loss: -9.3095 - val_accuracy: 0.0019 - val_auc_480: 0.0000e+00 - val_f1_m: 0.0065 - val_precision_m: 0.0588 - val_recall_m: 0.0035
Epoch 2/10
201/201 [==============================] - 1s 7ms/step - loss: -10.2257 - accuracy: 0.2141 - auc_480: 0.0000e+00 - f1_m: 0.5721 - precision_m: 0.9447 - recall_m: 0.4335 - val_loss: -10.0574 - val_accuracy: 0.2955 - val_auc_480: 0.0000e+00 - val_f1_m: 0.7132 - val_precision_m: 0.9749 - val_recall_m: 0.5736
Epoch 3/10
201/201 [==============================] - 2s 8ms/step - loss: -10.4930 - accuracy: 0.3202 - auc_480: 0.0

,Model,Arousal_Signal,Train_Study,Test_Study,Train_Accuracy,Test_Accuracy,Train_AUC,Test_AUC,Train_F1,Test_F1,Train_Recall,Test_Recall,Train_Precision,Test_Precision,Train_Specificity,Test_Specificity
0,LSTM,PP,sim1,sim2,0.356608,0.44721,0.849019,0.93241,0.418181,0.47757,0.356608,0.44721,0.505454,0.512353,0.0,0.0
1,LSTM,PP,sim1,tt1,0.356608,0.370837,0.849019,0.849124,0.418181,0.424133,0.356608,0.370837,0.505454,0.49532,0.0,0.0
2,LSTM,PP,sim1,office_tasks,0.356608,0.446938,0.849019,0.946965,0.418181,0.471969,0.356608,0.446938,0.505454,0.49997,0.0,0.0
3,LSTM,HR,sim1,sim2,0.381919,0.470872,0.865190,0.947918,0.437073,0.483619,0.381919,0.470872,0.510847,0.497075,0.0,0.0
4,LSTM,HR,sim1,tt1,0.381919,0.353247,0.865190,0.842578,0.437073,0.413299,0.381919,0.353247,0.510847,0.49795,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,LSTM,BR,sim1___sim2___tt1___office_tasks,None,0.498142,None,0.994894,None,0.500343,None,0.498142,None,0.502563,None,0.0,None
143,LSTM,PP_HR,sim1___sim2___tt1___office_tasks,None,0.452472,None,0.961531,None,0.469871,None,0.452472,None,0.488661,None,0.0,None
144,LSTM,HR_BR,sim1___sim2___tt1___office_tasks,None,0.489669,None,0.991996,None,0.493620,None,0.489669,None,0.497635,None,0.0,None
145,LSTM,PP_BR,sim1___sim2___tt1___office_tasks,None,0.493377,None,0.981508,None,0.502395,None,0.493377,None,0.511750,None,0.0,None


In [251]:
# loss: 0.0208 - accuracy: 0.9001 - f1_m: 0.8761 - precision_m: 0.9773 - recall_m: 0.8009
# val_loss: 0.0325 - val_accuracy: 0.8527 - val_f1_m: 0.8304 - vals_precision_m: 0.9588 - val_recall_m: 0.7407

In [252]:
modeling_time = end_time - start_time
print("--- Total Time %s mins (%s hours) ---" % (modeling_time//60, modeling_time//3600))

--- Total Time 54.0 mins (0.0 hours) ---


In [253]:
# root_path = add_path(all_studies_data_dir, metrics_dir) 
# files = ['metrics_lstm_v0.csv', 'metrics_lstm_v1.csv']
# final_file_name = 'metrics_lstm_all.csv'

# concat_df(root_path, files, final_file_name)

In [254]:
# start_time = time.time()

# for study_combination in study_combinations_to_run:
    
#     train_studies = study_combinations[study_combination]['Train']
#     test_studies = study_combinations[study_combination]['Test']
#     # print(train_studies, test_studies, study_combination)
    
#     train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
#     # print(all_studies_df.shape, train_df.shape)
    
#     model_metrics = {}

    
#     for arousal_signal in arousal_signals:
        
#         train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
#         if ground_truth_method == 'ecdf': 
#             train_df = train_df[train_df[arousal_signal+'_Arousal_Mode_Cat'] != -1]
        
#         try:
#             print(decorator + '\n' + decorator)
#             print('Train Studies: ', study_combination.upper() + ' & Modality: ' + arousal_signal)
#             print(decorator + '\n' + decorator + '\n')


#             # Ref: https://stackoverflow.com/questions/3975376/understanding-dict-copy-shallow-or-deep
#             model_metrics[arousal_signal] = {
#                 'K_Fold': copy.deepcopy(all_metrics),
#                 'Train': copy.deepcopy(all_metrics),
#                 'Test': {}
#             }

#             # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
#             # print(train_df.shape)
            
#             model = get_trained_dnn_model(train_df, arousal_signal)


#             # for i, test_study in enumerate(test_studies):
#             for j in range(1, len(test_studies)+1):
#                 for subset in itertools.combinations(test_studies, j):
              
#                     test_study_subset = list(subset)
#                     test_study_str = '___'.join(test_study_subset).upper()
#                     # print('\n------------------------ Test Study' + str(i+1) + ': ' + test_study.upper() + ' ------------------------')
#                     print('\n------------------------ Test Study: ' + test_study_str + ' ------------------------')
 
#                     model_metrics[arousal_signal]['Test'][test_study_str] = copy.deepcopy(all_metrics)
#                     test_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(test_study_subset)]
#                     if ground_truth_method == 'ecdf': 
#                         test_df = test_df[test_df[arousal_signal+'_Arousal_Mode_Cat'] != -1]
#                     test_dnn_model(model, test_study_str, test_df, arousal_signal)

#                     metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, test_study_str, model_metrics, k_fold=run_k_fold), ignore_index = True)

#             if not test_studies: metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, None, model_metrics), ignore_index = True)
    
#         except Exception as e:
#             print("An exception occurred for %s %s --> %s" % (study_combination, arousal_signal, e))
    
#     # pprint.pprint(model_metrics)
#     # overleaf_final_tables[study_combination] = get_overleaf_table(study_combination, model_metrics)
#     study_combination_model_metrics[study_combination] = model_metrics
    
    
# display(metrics_df)
# metrics_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + 'lstm_' + dnn_window_size + '_' + get_date_time() + '.csv', sep=',', index=False)


# end_time = time.time()